In [157]:
import pandas as pd
import numpy as np
from ambry import library
l = library()
p = l.get('ffiec.gov-hmda-2010e-lar-2012-ca').partition
all_df = p.pandas
df = all_df[all_df.county_code == 73]

In [ ]:
def add_race_eth(df):
    eth_map = {
        1: 'h',
        2: 'nh',
        3: None,
        4: None,
        5: None
    }

    race_map = {
        1: 'aian',
        2: 'as',
        3: 'b',
        4: 'pi',
        5: 'w',
        6: None,
        7: None,
        8: None
    }

    re_map = {
        ('h','aian'): 'h',    ('nh','aian'): 'aian', (None,'aian'): 'aian',
        ('h','as')  : 'h',    ('nh','as')  : 'as',   (None,'as')  : 'as',
        ('h','b')   : 'b',    ('nh','b')   : 'b',    (None,'b')   : 'b',
        ('h','pi')  : 'h',    ('nh','pi')  : 'pi',   (None,'pi')  : 'pi',
        ('h','w')   : 'h',    ('nh','w')   : 'w',    (None,'w')   : 'w',
        ('h',None)  : 'h',    ('nh',None)  : 'w',    (None,None)  : None,
    }

    df.loc[:,'race_eth'] = df.apply(lambda row: re_map[(eth_map[row.applicant_ethnicity], race_map[row.applicant_race_1])], axis=1)

add_race_eth(df)
dfs = df[['id','census_tract_number', 'applicant_ethnicity','applicant_race_1', 'race_eth']]
dfc = dfs.groupby(['census_tract_number', 'race_eth']).count()[['id']]
df_re = dfc.reset_index().pivot('census_tract_number', 'race_eth','id')
df_re


From : http://www.ffiec.gov/geocode/help3.aspx

Tract Income Level - This corresponds to tract classifications as defined by the HMDA and CRA regulations. This field is based on the Tract Median Family Income %:

If the Median Family Income % is < 50% and > 0 then the Income Level is Low.

If the Median Family Income % is >= 50% and < 80% then the Income Level is Moderate.

If the Median Family Income % is >= 80% and < 120% then the Income Level is Middle.

If the Median Family Income % is >=120% then the Income Level is Upper.

If the Median Family Income % is 0% then the Income Level is Not Known.

,applicant_income,hud_median_family_income,tract_to_msa_md_income
1480794,0239,75900,121.57
1480795,0038,75900,105.31
1480796,0082,75900,123.48
1480797,0032,75900,111.24
1480798,0131,75900,124.15


In [182]:
def add_income_group(df):
    import numpy as np
    
    ranges = {
        (0,.5): 'low',
        (.5,.8): 'mod',
        (.8,1.2): 'mid',
        (1.2,np.inf): 'high'
    }
    
    def ig(x):
        if x == 0:
            return 'na'
        elif 0.0 < x < .5:
            return 'low'
        elif 0.5 <= x < .8:
            return 'mod'
        elif 0.8 <= x < 1.2:
            return 'mid'
        elif 1.2 < x:
            return 'high'
        
    def to_float(v):
        try:
            return float(v)
        except:
            return 0
        
    
    df.loc[:,'income_group'] = df.apply(lambda row: ig( to_float(row.applicant_income) * 100 / row.hud_median_family_income) , 
                                        axis=1)

df.applicant_income = dfi.applicant_income.astype(float, raise_on_error=False)
add_income_group(df)

In [183]:
    
dfi = df[['applicant_income','hud_median_family_income', 'tract_to_msa_md_income','income_group']]
dfi.income_group.value_counts()
    

low     207208
na       28465
mod       4278
mid       1557
high      1198
dtype: int64